Inside __/pytorch/images__, you should find a __/style_images__ folder and a __/sample_content_images__ folder.

- Inside the __/style_images__ folder is an image of Vangogh painting (titled, __sample_vangogh.jpg__) that we will be using as our sample style image. 
- Inside the __/sample_content_images__ folder, which we will use solely for the purposes of testing locally, there are 4 sample images that we will use to apply the style onto. (images are titled, __sample_%1.jpg__)

The interactive notebook (__style_transfer_interactive.ipynb__) and the script (__style_transfer_script.py__) will using the above directories.

## (Optional) Tuning the style transfer hyperparameters interactively

The first thing we want to do is to test the style transfer scripts locally and make sure the hyper parameters are set appropriately. This will be done using the __style_transfer_interactive.ipynb__ notebook.

Open the notebook __/pytorch/style_transfer_interactive.ipynb__ and tune the variables following variables as desired: 

- STYLE_WEIGHT = 10**8
- CONTENT_WEIGHT = 10**0
- NUM_STEPS = 300

*the defaults shown above _tend_ to work nicely for most images.

## Testing the style transfer script locally

Lets make sure that our style transfer script is running correctly on our local machine. First we need to create the directory to store the output images.

In [1]:
%%bash
mkdir pytorch/images/sample_output_images

If you changed the hyperparameters in the above section and would like to apply it, you can use the following variables: `--style-weight`, `--content-weight`, and/or `--num-steps`

In [14]:
%%bash
cd pytorch &&
    python style_transfer_script.py \
    --style-image ./images/style_images/sample_vangogh.jpg \
    --content-image-dir ./images/sample_content_images \
    --content-image-list sample_0.jpg,sample_1.jpg \
    --output-image-dir ./images/sample_output_images \
    --style-weight 100000000 \
    --content-weight 1 \
    --num-steps 300 \
    --log-file 'sample_style_transfer_script'

2018-08-13 18:01:42,168 - __main__ - DEBUG - Images to process: 2
2018-08-13 18:01:42,168 - __main__ - DEBUG - GPU detected: True, image size: 512
2018-08-13 18:01:45,877 - __main__ - DEBUG - Time (in seconds) to load style image: 3.708443
2018-08-13 18:01:47,387 - __main__ - DEBUG - Time (in seconds) to load VGG19 model: 1.509391
2018-08-13 18:01:47,403 - __main__ - DEBUG - Time (in seconds) to apply style-transfer to batch of 2 images: 0.016245
2018-08-13 18:01:47,404 - __main__ - DEBUG - Average Time (in seconds) to apply style-transfer to each image: 0.008122


We can now inspect the output directory to make sure that the output images are there.

In [3]:
%%bash
ls pytorch/images/sample_output_images

Lets see what one of those images look like:

![sample_0](pytorch/images/sample_output_images/sample_1.jpg)

## Setup your video for batch style transfer

### Download Video: 
_TODO - update to use MSFT compliant video_

The first thing we need to do is to download a video that we would like to apply style transfer onto. To do so, we'll be using __youtube-dl__, a simple open source command line utility to download a video from YouTube. We'll keep the downloaded video in a new folder: __pytorch/video__. 

First we'll download __youtube-dl__ and use it to download a video. In this notebook, we're going to download a short one and a half minute video of chickens. In the download command, we'll also use the flag `-f 22` to tell __youtube_dl__ that we want to download an mp4 format with dimensions 720x1280. 

*Feel free to manually place your own video (mp4) file into that directory instead of using __youtube-dl__ to download one...

In [4]:
%%bash
mkdir pytorch/video

In [5]:
%%bash
sudo -H pip install --upgrade youtube-dl &&
    cd pytorch/video &&
    youtube-dl -f 22 -o "chicken.%(ext)s" "https://www.youtube.com/watch?v=D23sMvVnrow"

  Found existing installation: youtube-dl 2018.7.29
    Uninstalling youtube-dl-2018.7.29:
      Not removing or modifying (outside of prefix):
      /usr/bin/youtube-dl
      Successfully uninstalled youtube-dl-2018.7.29
[youtube] D23sMvVnrow: Downloading webpage
[youtube] D23sMvVnrow: Downloading video info webpage
[download] Destination: chicken.mp4
[download] 100% of 23.41MiB in 00:00


You are using pip version 8.1.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Pre-process video with ffmpeg

Next we need to use __ffmpeg__ to extract the audio file, which we will save as __chicken.aac__ under the video directory.

In [6]:
%%bash 
cd pytorch/video &&
    ffmpeg -i chicken.mp4 -vn -acodec copy chicken.aac

ffmpeg version 2.8.14-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.9) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --e

Finally, we need break up the frames of the video into separate individual images. The images will be saved inside a new folder under the images directory, called __chicken_frames__.

In [7]:
%%bash
cd pytorch/images/ &&
    mkdir chicken_frames && cd chicken_frames &&
    ffmpeg -i ../../video/chicken.mp4 %05d_chicken.jpg -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '../../video/chicken.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: isommp42
    creation_time   : 2017-09-23 09:38:19
  Duration: 00:01:39.61, start: 0.000000, bitrate: 1971 kb/s
    Stream #0:0(und): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(tv, bt709), 1280x720 [SAR 1:1 DAR 16:9], 1842 kb/s, 25 fps, 25 tbr, 90k tbn, 50 tbc (default)
    Metadata:
      creation_time   : 2017-09-23 09:38:19
      handler_name    : ISO Media file produced by Google Inc. Created on: 09/23/2017.
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 125 kb/s (default)
    Metadata:
      creation_time   : 2017-09-23 09:38:19
      handler_name    : ISO Media file produced by Google Inc. Created on: 09/23/2017.
[swscaler @ 0x26a9d60] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to '%05d_chicken.jpg':
  Metadata:
    major_brand     : mp42
    min

We can count the number of frames that the video produced:

In [8]:
%%bash
cd pytorch/images/chicken_frames && ls -1 | wc -l

2490


At this point, our file system should have the following under the folder __pytorch_style_transfer__:
```md
├── images/
│   ├── chicken_frames/ [<--new dir with video frames as images]
│   ├── sample_content_images/
│   ├── sample_output_images/
│   └── style_images/
├── video/
│   ├── chicken.mp4 [<--new video]
│   └── chicken.avi [<--new extracted audio file for video]
├── style_transfer_interactive.ipynb
└── style_transfer_script.py
```

Now that we have all the frames of the video stored seperately as images, we are ready to scale out and try scoring in the cloud.

## Testing our Style Transfer Script on Azure BatchAI

In this tutorial, we use Azure BatchAI to scale out of computation to multiple GPUs in Azure.

### Setting up your cluster

First, we need to create your cluster using the __azure/scripts/create_cluster.py__ script.

After running this command, go into the Azure portal and check that your cluster is successfully created.

In [9]:
%%bash 
python azure/scripts/create_cluster.py

Cluster state: steady; Allocated: 0; Idle: 0; Unusable: 0; Running: 0; Preparing: 0; Leaving: 0


Keyring cache token has failed: No recommended backend was available. Install the keyrings.alt package if you want to use the non-recommended backends. See README.rst for details.


### Upload files into Azure Blob Storage

Now we need to upload the script and model files to the fileshare using __az_copy__. We'll also upload our content images to test our Batch AI jobs. 

After running this command, go into the Azure portal or the Azure Storage Explorer and make sure that the files are correctly uploaded.

In [16]:
%%bash
azcopy \
    --source pytorch/style_transfer_script.py \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_INPUT_DIR}/${FS_SCRIPT_NAME} \
    --dest-key $STORAGE_ACCOUNT_KEY
    
azcopy \
    --source pytorch/images/style_images/sample_vangogh.jpg \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_INPUT_DIR}/${FS_STYLE_IMG_NAME} \
    --dest-key $STORAGE_ACCOUNT_KEY
    
azcopy \
    --source pytorch/images/chicken_frames \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_CONTENT_DIR} \
    --dest-key $STORAGE_ACCOUNT_KEY \
    --recursive

[2018/08/13 18:06:51][WARNING] The command line "--source "pytorch/images/chicken_frames" --destination "https://jsblobplayground.blob.core.windows.net/batchai/content" --recursive --dest-key ******" in the journal file "/home/jiata/Microsoft/Azure/AzCopy/AzCopy.jnl" is different from your input.
[2018/08/13 18:06:51][WARNING] Incomplete operation with different command line detected at the journal directory "/home/jiata/Microsoft/Azure/AzCopy".
[2018/08/13 18:06:52] Transfer summary:
-----------------
Total files transferred: 1
Transfer successfully:   1
Transfer skipped:        0
Transfer failed:         0
Elapsed time:            00.00:00:00
[2018/08/13 18:06:53] Transfer summary:
-----------------
Total files transferred: 1
Transfer successfully:   1
Transfer skipped:        0
Transfer failed:         0
Elapsed time:            00.00:00:00
[2018/08/13 18:06:55] Transfer summary:
-----------------
Total files transferred: 2490
Transfer successfully:   2490
Transfer skipped:        0

### Create a Test job

Finally, lets test that our style transfer script works on the Batch AI cluster. Use the __azure/scripts/create_job.py__ script to kick off a job. 

NOTE - this command could take a while to execute. Remember that this will apply style transfer onto each frame of the video - this means that the code will optimize the style transfer loss function for every frame. 

After running this command, go into the Azure portal under your BatchAI account to make sure the job is sucessfully created.

In [17]:
%%bash
python azure/scripts/create_job.py --job-batch-size 100

2018-08-13 18:08:17,334 - __main__ - DEBUG - Created job #0, named job0_08_13_2018_180759, with 100 images.
2018-08-13 18:08:34,850 - __main__ - DEBUG - Created job #1, named job1_08_13_2018_180817, with 100 images.
2018-08-13 18:08:51,533 - __main__ - DEBUG - Created job #2, named job2_08_13_2018_180834, with 100 images.
2018-08-13 18:09:09,173 - __main__ - DEBUG - Created job #3, named job3_08_13_2018_180851, with 100 images.
2018-08-13 18:09:26,249 - __main__ - DEBUG - Created job #4, named job4_08_13_2018_180909, with 100 images.
2018-08-13 18:09:42,504 - __main__ - DEBUG - Created job #5, named job5_08_13_2018_180926, with 100 images.
2018-08-13 18:09:58,835 - __main__ - DEBUG - Created job #6, named job6_08_13_2018_180942, with 100 images.
2018-08-13 18:10:15,412 - __main__ - DEBUG - Created job #7, named job7_08_13_2018_180958, with 100 images.
2018-08-13 18:10:32,120 - __main__ - DEBUG - Created job #8, named job8_08_13_2018_181015, with 100 images.
2018-08-13 18:10:48,525 - __

Keyring cache token has failed: No recommended backend was available. Install the keyrings.alt package if you want to use the non-recommended backends. See README.rst for details.


### Inspect the results

When the jobs finish running, you can use the Azure portal or Storage explorer to inspect the output images.

Inside your Blob Container, you should notice that a new directory with the datetime-stamp is created. Output images are stored there.

## Running it with Docker

After we've tested that the BatchAI jobs are successfully created, we now want to build a docker container and check that we can run the BatchAI job from a docker container.

First we need to build the docker image using the Dockerfile which will upload all Azure utility python files as well as the __create_job.py__ file into the image.

In [ ]:
%%bash
cd azure &&
    sudo docker build -t $DOCKER_IMAGE .

Then we need to test that the BatchAI job can be executed from the docker image we just built. Because the __create_job.py__ file requires many environment variables, we will use the __docker_run.sh__ script to pass in all the required environment variables for the image to successfully run locally. 

In [ ]:
%%bash
cd azure &&
    source docker_run.sh -t $DOCKER_IMAGE

Finally, we have to publish the image to Dockerhub. _Make sure you replace <your-dockerhub-username>_ with your Dockerhub username. Don't forget that you'll have to make sure you're already logged in before being able to push your image up. (`docker login`)

In [ ]:
%%bash
cd azure &&
    sudo docker tag bai_job $DOCKER_USERNAME:$DOCKER_IMAGE &&
    sudo docker push $DOCKER_USERNAME:$DOCKER_IMAGE

## Setting up ACI with Logic Apps

First we need to generate the ARM deployment template for deploying ACI and Logic App to set up the trigger. We will use the __generate_trigger_arm.py__ script to generate an ARM Template (JSON) which we can use to deploy the ACI and Logic App. The script uses __azure/deployments/template.trigger_arm.json__ to generate __azure/deployments/trigger_arm.json__, which we can use to execute the deployment.

If you inspect the ARM Template, you'll see that Logic App will be triggered when a new file is added to the blob. It will then use the contents of the blob and look for a directory in that Blob Container and use it as the content image file directory for the transfer.

For example, if we upload a file, titled `foo.txt`, with the content "content_image_dir", the Logic App will look for a directory in the Blob Container titled "content_image_dir", and start to process the images it finds in the directory.

First lets generate the trigger template:

In [8]:
%%bash
cd azure/deployments &&
    python ../scripts/generate_trigger_arm.py trigger_arm.json

Now lets use the __az cli__ to deploy the solution.

In [ ]:
%%bash
az group deployment create \
    --name aci_logicapp_deployment \
    --resource-group $RESOURE_GROUP \
    --template-file azure/deployments/trigger_arm.json

## Trigger the process

Finally we need to trigger the process by loading something into blob. Normally, this can be done with any process, but for the purposes of this demo, lets do so simply with __az_copy__.

First we create a file, __foo.txt__ with the contents being the name of the directory in the blob that we want to apply style transfer too.

In [ ]:
%%bash
touch foo.txt &&
    echo $FS_CONTENT_DIR > foo.txt

Next we have to copy that file over to storage to trigger the process:

In [ ]:
%%bash
azcopy \
    --source foo.txt \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/foo.txt \
    --dest-key $STORAGE_ACCOUNT_KEY

At this point, we can inspect the Azure portal to see all the moving parts:
- Logic Apps will be triggers and will spin up ACI
- ACI will break up the content images in blob and create BatchAI jobs
- The BatchAI cluster will scale up and start processing the work
- As the style transfer script is executed in batch on BatchAI, we will see the completed images (as well as logs) saved back to blob 

## Download Results and Re-stitch Video

The last step is to download and restitch the video so that we can enjoy the same video, but now with each frame with style transfer applied.

First, we have to download the frames:

In [ ]:
%%bash
cd pytorch/images &&
    azcopy \
        --source https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME} \
        --destination . \
        --source-key $STORAGE_ACCOUNT_KEY \
        --include "output_content_08_08_2018_162725" \
        --recursive

Then lets use ffmpeg to stitch independent frames back into a video.

In [ ]:
%%bash
cd pytorch &&
    ffmpeg \
        -framerate 30 \
        -i images/"output_content_08_08_2018_162725"/%05d_chicken.jpg \
        -c:v libx264 \
        -profile:v high -crf 20 -pix_fmt yuv420p \
        video/chicken_processed.mp4 